<a href="https://akademie.datamics.com/kursliste/">![title](bg_datamics_top.png)</a>

<center><em>© Datamics</em></center><br><center><em>Besuche uns für mehr Informationen auf <a href='https://akademie.datamics.com/kursliste/'>www.akademie.datamics.com</a></em>

# CamShift Tracking

In [ ]:
import numpy as np
import cv2 

# Zeichne das Video auf
cap = cv2.VideoCapture(0)

# Nimm das erste Frame (Einzelbild) des Streams
ret,frame = cap.read()


# Richte das initiale Tracking-Fenster ein


# Wir detektieren zuerst das Gesicht und legen es als unseren Startbereich fest
face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame) 

# Konvertiere diese Liste mit einem einzelnem Array zu einem Tupel (x,y,w,h)
(face_x,face_y,w,h) = tuple(face_rects[0]) 
track_window = (face_x,face_y,w,h)
# Finde die ROI zum Verfolgen
roi = frame[face_y:face_y+h, face_x:face_x+w]


# Verwende das HSV-Farbschema
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Finde das Histogramm, um das Ziel auf jedem Frame zurückzuprojizieren zur Berechnung der Mittelwertverschiebung
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

# Normalisiere die Arraywerte des Histogramms für ein gegebenes Minimum von 0 und Maximum von 255
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)


# Richte ein Kriterium zum Beenden ein, entweder 10 Iterationen oder eine Bewegung von mindestens 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    ret ,frame = cap.read()
    if ret == True:
        
        # Nimm das Frame in HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Berechne die Rückprojektion basierend auf dem vorher erzeugten roi_hist
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        #########################################################
        #########################################################
        ########## CAM SHIFT ####################################
        ########################################################
        #######################################################
        
        # Wende Camshift an, um die neuen Koordinaten des Rechtecks zu erhalten
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        
        # Zeichne es auf das Bild
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, (0,0,255),5)
        cv2.imshow('img2',img2)
        
        ########################################################
        #######################################################
        ########################################################
        #######################################################
        
        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
        
    else:
        break
        
cv2.destroyAllWindows()
cap.release()